# Cleaning and Formatting the Data

##### The purpose of this notebook is to read in the raw data in various formats and combine it into one master csv

In [67]:
import pandas as pd
import json
import os
import datetime

### Part 1: Make Master DataFrame

First raw data was a csv called "misc_queries.csv" so let's start with that as the baseline

In [2]:
q = pd.read_csv('misc_queries.csv',index_col=False)

In [3]:
q.head()

,Query,Source,Date Added,(Num Occurances)
0,michelle wu husband,starting,3/30/2021 0:00,NaN
1,andrea campbell husband,starting,4/1/2021 0:00,NaN
2,dana depelteau husband,starting,4/3/2021 0:00,NaN
3,annisa essaibi george husband,starting,4/4/2021 0:00,NaN
4,jon santiago wife,starting,4/6/2021 0:00,NaN


In [4]:
#note that the Num Occurances was a empty column that we didn't use, so let's drop that
q.drop(columns=['(Num Occurances)'],inplace=True)
q.head()

,Query,Source,Date Added
0,michelle wu husband,starting,3/30/2021 0:00
1,andrea campbell husband,starting,4/1/2021 0:00
2,dana depelteau husband,starting,4/3/2021 0:00
3,annisa essaibi george husband,starting,4/4/2021 0:00
4,jon santiago wife,starting,4/6/2021 0:00


Since we want to do some analysis by city, the next thing is to make a column in the dataframe with the city that the query is from. First, we'll need the lists of candidates from each city.

In [68]:
cands = pd.read_csv('candidates.csv',index_col=False)
cands_bos = cands[cands.city=='Boston'].name.to_list()
cands_nyc = cands[cands.city=='New York City'].name.to_list()

In [87]:
cands_bos

['michelle wu',
 'andrea campbell',
 'dana depelteau',
 'annissa essaibi george',
 'jon santiago',
 'kim janey',
 'michael bianchi',
 'john barros']

In [62]:
def add_city(query):
    """looks at query and identifies which city it is part of"""
    if 'new york' in query.lower(): return 'NYC'
    elif 'boston' in query.lower(): return 'Boston'
    
    for name in cands_bos: 
        if name in query: return 'Boston'
    return 'NYC'

q['city'] = q['Query'].apply(add_city)

In [64]:
q[]

,Query,Source,Date Added,city,seed_query,candidate,gender
22,jon santiago races 2021,autocomplete,2021-04-27,Boston,jon santiago race,jon santiago,M
23,isaac wright jr daughter instagram,autocomplete,2021-04-27,NYC,isaac wright jr,isaac wright jr,M
24,john barros wiki,autocomplete,2021-04-27,Boston,john barros,john barros,M
25,scott stringer backgrounds,autocomplete,2021-04-27,NYC,scott stringer background,scott stringer,M
26,art chang mayor election results 2018,autocomplete,2021-04-27,NYC,art chang,art chang,M
...,...,...,...,...,...,...,...
188789,nyc mayoral election 2021,related searches,2021-11-09,NYC,,None,
188790,nyc mayoral race 2021 results,related searches,2021-11-09,NYC,,None,
188791,nyc primary candidates 2021,related searches,2021-11-09,NYC,,None,
188792,2013 nyc mayoral election,related searches,2021-11-09,NYC,,None,


Let's also fix the date added column, because there seem to be some inconsistencies

In [8]:
#fix date column format
def fix_date(x):
    if x[4] == '-':
        return datetime.datetime.strptime(x,"%Y-%m-%d")
    else:
        return datetime.datetime.strptime(x[:-5],"%m/%d/%Y")
    
q['Date Added'] = q['Date Added'].apply(fix_date)
q['Date Added'].unique()

array(['2021-03-30T00:00:00.000000000', '2021-04-01T00:00:00.000000000',
       '2021-04-03T00:00:00.000000000', '2021-04-04T00:00:00.000000000',
       '2021-04-06T00:00:00.000000000', '2021-04-08T00:00:00.000000000',
       '2021-04-10T00:00:00.000000000', '2021-04-12T00:00:00.000000000',
       '2021-04-13T00:00:00.000000000', '2021-04-15T00:00:00.000000000',
       '2021-04-16T00:00:00.000000000', '2021-04-18T00:00:00.000000000',
       '2021-04-19T00:00:00.000000000', '2021-04-20T00:00:00.000000000',
       '2021-04-21T00:00:00.000000000', '2021-04-22T00:00:00.000000000',
       '2021-04-23T00:00:00.000000000', '2021-04-24T00:00:00.000000000',
       '2021-04-25T00:00:00.000000000', '2021-04-26T00:00:00.000000000',
       '2021-04-27T00:00:00.000000000', '2021-04-29T00:00:00.000000000',
       '2021-04-30T00:00:00.000000000', '2021-05-01T00:00:00.000000000',
       '2021-05-02T00:00:00.000000000', '2021-05-03T00:00:00.000000000',
       '2021-05-04T00:00:00.000000000', '2021-05-05

We need to combine this with the data that Emma has, which is in json format

In [9]:
raw = 'audit-suggestions-data'
def add_queries_to_df(city, dataframe):
    base = raw+f'/{city}/' #the city names are two separate folders 
    for f in os.listdir(raw+'/'+city):
        with open(raw+f'/{city}/'+f,encoding='utf-8') as fp:
            try:
                data = json.load(fp)
            except Exception as e:
                print(e, raw+f'/{city}/'+f)

        qdict = {} #formatting this to add to the dataframe easier
        #add the type of query
        if 'autocomplete' in f: qdict['Source'] = 'autocomplete'
        else: qdict['Source'] = 'related searches'
        #add the date in correct format
        qdict['Date Added'] = datetime.datetime.strptime(f[:f.find('_')],"%Y-%m-%d")
        qdict['city'] = city


        for seed_query in data.keys():
            #the value of the key is the list of queries stemming from that seed query
            
            if type(data[seed_query]) != list: continue
            for query in data[seed_query]:
                qdict['Query'] = query
                dataframe = dataframe.append(qdict,ignore_index=True)
    return dataframe

In [10]:
print(len(q))
q = add_queries_to_df("Boston",q)
print(len(q))
q = add_queries_to_df("NYC",q)
print(len(q))

4133
Expecting value: line 1 column 1 (char 0) audit-suggestions-data/Boston/2021-08-07_autocomplete.json
Expecting value: line 1 column 1 (char 0) audit-suggestions-data/Boston/2021-11-01_related_searches.json
154338
Expecting value: line 1 column 1 (char 0) audit-suggestions-data/NYC/2021-08-17_autocomplete.json
188794


In [71]:
q

,Query,Source,Date Added,city,seed_query,candidate,gender
22,jon santiago races 2021,autocomplete,2021-04-27,Boston,jon santiago race,jon santiago,M
23,isaac wright jr daughter instagram,autocomplete,2021-04-27,NYC,isaac wright jr,isaac wright jr,M
24,john barros wiki,autocomplete,2021-04-27,Boston,john barros,john barros,M
25,scott stringer backgrounds,autocomplete,2021-04-27,NYC,scott stringer background,scott stringer,M
26,art chang mayor election results 2018,autocomplete,2021-04-27,NYC,art chang,art chang,M
...,...,...,...,...,...,...,...
188789,nyc mayoral election 2021,related searches,2021-11-09,NYC,,None,
188790,nyc mayoral race 2021 results,related searches,2021-11-09,NYC,,None,
188791,nyc primary candidates 2021,related searches,2021-11-09,NYC,,None,
188792,2013 nyc mayoral election,related searches,2021-11-09,NYC,,None,


In [11]:
#let's save this as a csv so that we don't have to run it again - it took a while!
q.to_csv('all_queries.csv',index=False)

### Part 2: Add Helper Info for Analysis

Now that we have this baseline master csv, we want to add some information like gender, seed query stem, etc. and also separate related searches from autocomplete searches to make analysis easier

In [24]:
#Before starting, let's remove the seed queries from the dataframe
q = q[q.Source != 'starting']

##### Subpart 2A: Find seed query that each query stemmed from
To do this, we'll first need to generate all the seed queries. Since our seed query list was a template with [candidate] as a placeholder for the name, let's grab of lists of candidates and generate the seed queries

In [72]:
cands_bos_w = ['michelle wu', 'andrea campbell', 'annissa essaibi george', 'kim janey']
cands_bos_m = list(set(cands_bos) - set(cands_bos_w))
cands_nyc_w = ['maya wiley','kathryn garcia','dianne morales']
cands_nyc_m = list(set(cands_nyc) - set(cands_nyc_w))

In [26]:
#read in the seed queries 
s = pd.read_excel('seed_queries.xlsx', index_col=False)

Helper function that generates a list of queries for all candidates in a particular city

In [73]:
def generate_queries(city, cands):
    s = pd.read_excel('seed_queries.xlsx', sheet_name=f'{city} Seed Queries', index_col=False)
    seed_queries = s[s.Source=='seed'].Query.tolist()[2:] #exclude the 2 that are not about a specific candidate
    queries = []
    for query in seed_queries:
        queries.extend(query.replace('[candidate]',candidate) for candidate in cands)
    return queries

In [74]:
queries_bos = generate_queries('Boston',cands_bos)
queries_nyc = generate_queries('NYC',cands_nyc)

Now that we have these lists, we can go through the queries in our dataframe and add the seed query that the query resulted from

In [75]:
def find_seed(query, source, city):
    if source == 'autocomplete':
        if city =='Boston':
            for seed_q in list(reversed(queries_bos)): #reverse so it doesn't just get cand name first 
                if seed_q in query:
                    return seed_q
        else:
            for seed_q in list(reversed(queries_nyc)): #reverse so it doesn't just get cand name first 
                if seed_q in query:
                    return seed_q
    return ''

q['seed_query'] = q.apply(lambda x: find_seed(x['Query'], x['Source'], x['city']),axis=1)

In [76]:
q.seed_query.unique()

array(['jon santiago race', 'isaac wright jr', 'john barros',
       'scott stringer background', 'art chang', '', 'andrew yang',
       'curtis sliwa nyc', 'aaron foldenauer', 'curtis sliwa',
       'eric adams', 'art chang mayoral election', 'ray mcguire',
       'dianne morales', 'kathryn garcia', 'michelle wu for mayor',
       'shaun donovan', 'andrew yang net worth',
       'dana depelteau mayoral election', 'andrea campbell',
       'paperboy prince endorsement', 'andrea campbell race',
       'dana depelteau', 'andrew yang for mayor', 'kim janey boston',
       'michelle wu boston', 'john barros boston',
       'andrea campbell boston', 'art chang race',
       'andrew yang background', 'michael bianchi', 'scott stringer',
       'art chang endorsement', 'dianne morales net worth',
       'art chang partner', 'ray mcguire partner', 'kim janey',
       'scott stringer race', 'scott stringer for mayor', 'jon santiago',
       'michelle wu', 'andrew yang nyc', 'john barros race', 

In [77]:
q.head()

,Query,Source,Date Added,city,seed_query,candidate,gender
22,jon santiago races 2021,autocomplete,2021-04-27,Boston,jon santiago race,jon santiago,M
23,isaac wright jr daughter instagram,autocomplete,2021-04-27,NYC,isaac wright jr,isaac wright jr,M
24,john barros wiki,autocomplete,2021-04-27,Boston,john barros,john barros,M
25,scott stringer backgrounds,autocomplete,2021-04-27,NYC,scott stringer background,scott stringer,M
26,art chang mayor election results 2018,autocomplete,2021-04-27,NYC,art chang,art chang,M


##### Subpart 2B: Add the name of the candidate that the query is about as well as the gender of that candidate

In [78]:
def add_cand(query, city):
    if city =='Boston':
        for candidate in cands_bos:
            if candidate in query: return candidate
    else:
        for candidate in cands_nyc:
            if candidate in query: return candidate      

def get_gender(candidate, city):
    if city =='Boston':
        if candidate in cands_bos_w: return 'W'
        elif str(candidate) == 'None': return ''
        else: return 'M'
    else:
        if candidate in cands_nyc_w: return 'W'
        elif str(candidate) == 'None': return ''
        else: return 'M'

q['candidate'] = q.apply(lambda x: add_cand(x['Query'], x['city']),axis=1)
q['gender'] = q.apply(lambda x: get_gender(x['candidate'], x['city']),axis=1)

In [97]:
q[(q.candidate.isnull())&(q.Query.str.contains('george'))].Query.unique()

array(['annissa essaibi george bio', 'annissa essaibi george race 2020',
       'annissa essaibi george lgbt', 'annissa essaibi george',
       'annissa essaibi george mayoral election 2021',
       'annissa essaibi george mayoral election 2019 candidates',
       'annissa essaibi george net worth 2018',
       'annissa essaibi george husband',
       'annissa essaibi george mayoral election 2018',
       'annissa essaibi george instagram',
       'annissa essaibi george education',
       'annissa essaibi george campaign manager',
       'annissa essaibi george for mayor', 'annissa essaibi george mayor',
       'annissa essaibi george family',
       'annissa essaibi george race 2021',
       'annissa essaibi george net worth forbes',
       'annissa essaibi george endorsements',
       'annissa essaibi george mayoral election 2018 candidates',
       'annissa essaibi george mayoral elections 2021',
       'annissa essaibi george twitter',
       'annissa essaibi george mayoral electi

In [89]:
q[q.gender=='W'].candidate.unique()

array(['dianne morales', 'kathryn garcia', 'michelle wu',
       'andrea campbell', 'kim janey', 'maya wiley',
       'annissa essaibi george'], dtype=object)

In [84]:
q[q.gender=='M'].candidate.unique()

array(['jon santiago', 'isaac wright jr', 'john barros', 'scott stringer',
       'art chang', 'andrew yang', 'curtis sliwa', 'aaron foldenauer',
       'eric adams', 'ray mcguire', 'shaun donovan', 'dana depelteau',
       'paperboy prince', 'michael bianchi', 'fernando mateo'],
      dtype=object)

In [85]:
q = q.replace('annisa','annissa',regex=True)

Now let's save this dataframe for future use!

In [86]:
q.to_csv('all_queries_with_data.csv',index=False)

### Part 3: Create Dataframes of just Related and Autocomplete Searches

In [43]:
q = pd.read_csv('all_queries_with_data.csv',index_col=False)

In [44]:
q[q.Source=='related searches'].to_csv('all_queries_related.csv',index=False)

In [46]:
q[q.Source=='autocomplete'].to_csv('all_queries_autocomplete.csv',index=False)